https://deep-learning-study.tistory.com/686

https://stackoverflow.com/questions/71818652/attributeerror-module-torchtext-has-no-attribute-legacy
torchtext import 안될때 버전변경

https://codlingual.tistory.com/91

https://proceedings.neurips.cc/paper/2014/file/a14ac55a4f27472c5d894ec1c3c743d2-Paper.pdf

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.legacy.datasets import Multi30k#stack-overflow 형님들은 신임
from torchtext.legacy.data import Field, BucketIterator
import spacy
import numpy as np
import random
import math
import time

In [2]:
seed=1234

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)#pytorch의 randomseed고정
torch.cuda.manual_seed(seed)#pytorch randomseed 고정
torch.backends.cudnn.deterministic=True#cudnn randomseed 를 고정

python -m spacy download en

python -m spacy download de

In [3]:
import de_core_news_sm
import en_core_web_sm

spacy_en= en_core_web_sm.load()
spacy_de=de_core_news_sm.load()

def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]#reverse

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

SRC=Field(tokenize=tokenize_de, init_token='<sos>', eos_token='<sos>', lower=True)
TRG=Field(tokenize=tokenize_en, init_token='<sos>', eos_token='<sos>', lower=True)

C:\Users\Lee_JongWoo\anaconda3\lib\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value [] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(


In [4]:
train_data, valid_data, test_data=Multi30k.splits(exts=('.de','.en'),fields=(SRC,TRG))
# train, validation, test 데이터를 불러오고, 다운로드 합니다.
# Multi30k dataset을 사용하여, 30,000개의 영어, 독일, 프랑스어 문장을 포함합니다.

In [5]:
print(vars(test_data.examples[0]))#입력 src는 입력단어가 거꾸로

{'src': ['.', 'anstarrt', 'etwas', 'der', ',', 'hut', 'orangefarbenen', 'einem', 'mit', 'mann', 'ein'], 'trg': ['a', 'man', 'in', 'an', 'orange', 'hat', 'starring', 'at', 'something', '.']}


In [6]:
print(vars(train_data.examples[0]))

{'src': ['.', 'büsche', 'vieler', 'nähe', 'der', 'in', 'freien', 'im', 'sind', 'männer', 'weiße', 'junge', 'zwei'], 'trg': ['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.']}


In [7]:
print(f'Number of training examples:{len(train_data.examples)}')
print(f'Number of validation examples:{len(valid_data.examples)}')
print(f'Number of testing examples:{len(test_data.examples)}')

Number of training examples:29000
Number of validation examples:1014
Number of testing examples:1000


In [8]:
SRC.build_vocab(train_data, min_freq=2)
TRG.build_vocab(train_data,min_freq=2)
#토큰화 전처리가 끝나면, 각단어에 고유한 정수를 맵핑해주는 정수 인코딩작업이 필요하다. 그리고 이 전처리를 위해서는 단어집합이 필요한데, 
#정의한 filed에 build_vocab을 이용하면 단어집합을 만들수 있다.
# min_freq=2는 2번 이상 등장한 토큰을 출력합니다.
# 토큰이 1번만 등장했다면 <unk>로 대체합니다.
#vocab은 training set에서만 만들어져야 함 (validation/test set에서 만들면 안 됨)

In [9]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

batch_size=128
train_iterator, valid_iterator, test_iterator=BucketIterator.splits((train_data, valid_data, test_data),batch_size=batch_size,device=device)

In [10]:
train_iterator

In [11]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim=hid_dim
        self.n_layers=n_layers
        #임베딩:입력값을 emb_dim벡터로 변경
        self.embedding=nn.Embedding(input_dim, emb_dim)
        
        #embeding을 입력받아 hid_dim크기의 hidden state,cell출력
        self.rnn=nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout)
        
        self.dropout=nn.Dropout(dropout)
    
    def forward(self,src):
        #source의 줄임말로 입력 문장
        #src=[src.len,batch_size]
        embedded= self.dropout(self.embedding(src))
        #initial hidden state는 제로텐서
        outputs, (hidden,cell)=self.rnn(embedded)
        # output: [src_len, batch_size, hid dim * n directions] 맨 위 레이어에서 각 time-stamp마다의 은닉 상태들
        # hidden: [n layers * n directions, batch_size, hid dim] 각 레이어의 마지막 은닉상태, h_T //z_t=h_T+c_T 이므로 레이어2개 곱
        # cell: [n layers * n directions, batch_size, hid dim] 각 레이어의 마지막 cell state, c_T
        return hidden, cell

In [12]:
class Decoder(nn.Module):
    def __init__(self, out_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim=output_dim
        self.hid_dim=hid_dim
        self.n_layers=n_layers
        
        #context vector를 받아  embdim으로 출력
        self.embedding=nn.Embedding(output_dim,emb_dim)
        self.rnn=nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout)
        self.fc_out=nn.Linear(hid_dim, output_dim)
        
        self.dropout=nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        # context = [n layers, batch size, hid dim]
        input= input.unsqueeze(0) #입력 [1,batchsize] 첫번째 input 은 <sos>
        ##input = [batch size] → unsqueeze → input = [1, batch size]
        ##embedded = [1, batch size, emb dim]--->input = [1, batch size] → embedding → dropout → embedded = [1, batch size, emb dim]
        embedded=self.dropout(self.embedding(input))
        
        output, (hidden,cell)= self.rnn(embedded, (hidden,cell))
        ##embedded, hidden, cell → rnn → output, hidden, cell
        # Decoder에서 항상 seq len = n directions = 1 
        # 한 번에 한 토큰씩만 디코딩하므로 seq len = 1
        # 따라서 output = [1, batch size, hid dim]
        # hidden = [n layers, batch size, hid dim]
        # cell = [n layers, batch size, hid dim]
        prediction=self.fc_out(output.squeeze(0))
        ##output = [1, batch size, hid dim] → squeeze → output = [batch size, hid dim]
        # prediction = [batch size, output dim]
        return prediction, hidden, cell

In [13]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder=encoder
        self.decoder=decoder
        self.device=device
         # encoder와 decoder의 hid_dim이 일치하지 않는 경우 에러메세지
        assert encoder.hid_dim == decoder.hid_dim, \
            'Hidden dimensions of encoder decoder must be equal'
         # encoder와 decoder의 hid_dim이 일치하지 않는 경우 에러메세지
        assert encoder.n_layers == decoder.n_layers, \
            'Encoder and decoder must have equal number of layers'
    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        #target word(Ground Truth)를 디코더의 다음 입력으로 넣어주는 기법
        #src=[src len,batch_size]
        #trg=[trg len,batch_size]
        
        batch_size=trg.shape[1]
        trg_len=trg.shape[0] # 타겟 토큰 길이 얻기
        trg_vocab_size=self.decoder.output_dim # context vector의 차원
        # decoder의 output을 저장하기 위한 tensor
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        # initial hidden state, Encoder의 마지막 은닉 상태가 Decoder의 초기 은닉상태로 쓰임
        hidden, cell=self.encoder(src)
        # Decoder에 들어갈 첫 input은 <sos> 토큰
        input=trg[0,:]
        # range(0,trg_len)이 아니라 range(1,trg_len)인 이유 : 0번째 trg는 항상 <sos>라서 그에 대한 output도 항상 0 
        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            outputs[t]=output# prediction 저장
            teacher_force=random.random()<teacher_forcing_ratio# teacher forcing을 사용할지, 말지 결정
            top1=output.argmax(1)# 가장 높은 확률을 갖은 값 얻기
            input=trg[t] if teacher_force else top1# teacher forcing의 경우에 다음 lstm에 target token 입력
            
        return outputs

In [14]:
input_dim=len(SRC.vocab)
output_dim=len(TRG.vocab)
enc_emb_dim=256
dec_emb_dim=256
hid_dim=512
n_layers=2
enc_dropout=0.5
dec_dropout=0.5
enc=Encoder(input_dim, enc_emb_dim, hid_dim, n_layers, enc_dropout)
dec=Decoder(output_dim, dec_emb_dim, hid_dim, n_layers, dec_dropout)

model=Seq2Seq(enc, dec, device).to(device)

In [15]:
#(-0.08, 0.08) 범위의 정규분포에서 모든 가중치를 초기화 
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform(param.data, -0.08,0.08)

model.apply(init_weights)

C:\Users\Lee_JongWoo\AppData\Local\Temp\ipykernel_14452\1057165369.py:4: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  nn.init.uniform(param.data, -0.08,0.08)


Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7852, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5892, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=5892, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [16]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {count_parameters(model):,} trainableparameters')

The model has 13,897,476 trainableparameters


In [17]:
optimizer=optim.Adam(model.parameters())


# loss function
# <pad> 토큰의 index를 넘겨 받으면 오차 계산하지 않고 ignore하기
# <pad> = padding
trg_pad_idx = TRG.vocab.stoi[TRG.pad_token]
criterion=nn.CrossEntropyLoss(ignore_index=trg_pad_idx)

In [18]:
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss=0
    
    for i, batch in enumerate(iterator):
        src=batch.src
        trg=batch.trg
        optimizer.zero_grad()
        
        output=model(src,trg)
        # trg = [trg len, batch size]
        # output = [trg len, batch size, output dim]
        output_dim=output.shape[-1]
        output=output[1:].view(-1, output_dim)#flatten 같은느낌
        trg = trg[1:].view(-1)
        loss=criterion(output,trg)
        loss.backward()
        
        # 기울기 폭발 막기 위해 clip
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        epoch_loss+=loss.item()
    
    return epoch_loss/len(iterator)
        

In [19]:
def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss=0
    
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            src=batch.src
            trg=batch.src
            
            # teacher_forcing_ratio = 0 (아무것도 알려주면 안 됨)
            output=model(src, trg, 0)
            # trg = [trg len, batch size]
            # output = [trg len, batch size, output dim]
            output_dim=output.shape[-1]
            output=output[1:].view(-1,output_dim)#flatten 같은 느낌? index 1부터 시작
            trg=trg[1:].view(-1)#얘도
            
             # trg = [(trg len - 1) * batch size]
            # output = [(trg len - 1) * batch size, output dim]
            
            loss=criterion(output, trg)
            epoch_loss+=loss.item()
            
    return epoch_loss/len(iterator)

In [20]:
def epoch_time(start_time, end_time):
    elapsed_time=end_time-start_time
    elapsed_mins=int(elapsed_time/60)
    elapsed_secs=int(elapsed_time- (elapsed_min*60))
    return elapsed_mins, elapsed_secs

In [21]:
# 학습 시작
num_epochs = 10
clip = 1

best_valid_loss = float('inf')

for epoch in range(num_epochs):
   
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, clip)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

KeyboardInterrupt: 

In [ ]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss=evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f}')